## Embedding

If atomic map numbers of donor atoms are correct (no repeats, lies in range \[1,6\]/\[1,4\] for OH/SP geometry, etc.), than one can generate 3D atomic coordinates:

In [1]:
# imports
import mace, py3Dmol

# view 3D structures
def view_complex(X, confId = 0):
    '''Shows molecule corresponding to the SMILES'''
    view = py3Dmol.view()
    view.addModel(X.ToXYZBlock(confId = confId), 'xyz')
    view.setStyle({'stick': {'radius': 0.15}, 'sphere': {'scale': 0.3}})
    view.setBackgroundColor('white')
    view.zoomTo()
    view.show()

In [2]:
# get 3D after initializing complex
ligands = ['[NH2:1]CC[NH2:2]', '[NH2:3]CC[NH2:4]', '[Cl-:5]', '[Cl-:6]']
X = mace.ComplexFromLigands(ligands = ligands, CA = '[Ru+2]', geom = 'OH')
X.AddConformer()
view_complex(X)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

If the atomic map numbers are set incorrectly, one must first generate possible stereomers (see the "Stereomers generation" section):

In [3]:
# try to get 3D coords without stereo info for CA
ligands = ['[NH2:1]CC[NH2:1]', '[NH2:1]CC[NH2:1]', '[Cl-:1]', '[Cl-:1]']
X1 = mace.ComplexFromLigands(ligands = ligands, CA = '[Ru+2]', geom = 'OH')
try:
    X1.AddConformer()
except Exception as e:
    print(e)

Bad SMILES: isotopic labels are not unique

The initial SMILES contains insufficient or erroneous info
on the positions of the ligands around the central atom
encoded with isotopic labels.

To use 3D generation and other features, first generate
possible stereomers



The main parameters of `Complex.AddConformers` method are number of conformers to generate `numConfs` and minimal RMSD between conformers when they are considered as identical `rmsThresh`:

In [4]:
# get several conformers with rms threshold
ligands = ['C[P:2](C)CCCC[N:3](C)CCCC[P:4](C)C', '[C-:1]#[O+]', '[C-:5]#[O+]', '[C-:6]#[O+]']
X = mace.ComplexFromLigands(ligands = ligands, CA = '[Ru+2]', geom = 'OH')
X.AddConformers(numConfs = 10, rmsThresh = 2.0)
X.GetNumConformers()

4

As one can see, number of generated conformers is less than `numConfs` value due to the filtering of similar conformations.

Analysis of molecular mechanical energies help to identify "good" and "bad" (potentially unsuitable for QM computations) geometries:

In [5]:
# get conf energies
Es = [X.GetConfEnergy(idx) for idx in range(X.GetNumConformers())]
Es

[154.16904500164776,
 153.00520591073658,
 160.57425249049072,
 159.33188783509556]

In [6]:
# find indexes of best/worst conformers
idx_min = X.GetMinEnergyConfId(0)
idx_max = X.GetMinEnergyConfId(X.GetNumConformers() - 1)
idx_min, idx_max

(1, 2)

In [7]:
# view
print('Best conformer:')
view_complex(X, confId = idx_min)
print('Worst conformer:')
view_complex(X, confId = idx_max)

Best conformer:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Worst conformer:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Generated conformations can be saved to the `xyz` files, which can be used futher in various computational pipelines.